In [1]:
import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext()

### Our Data

Reading in the Small example curated [at this site](https://lintool.github.io/Cloud9/docs/exercises/pagerank.html).

In [2]:
rdd = sc.textFile('../data/pagerank.txt')

Each row of data has a page in the 0th index, and all of the pages it links to in the rest of the list.

In [3]:
pages = rdd.map(lambda x: x.split())
pages.take(5)

[['9465097', '12566713', '11158207', '11145916', '11883199', '12857908'],
 ['11581419', '11287582', '9420209', '8709207', '11160736', '12610128'],
 ['8553535', '8709207', '12518224', '11044077', '9650960', '11886254'],
 ['7778283', '8709207', '8601783', '7494284', '9298727', '8973339'],
 ['11044077', '12518224', '9568045', '8553535', '9784363', '9650960']]

Sure enough, it has 93 rows. One for each page in the dataset.

In [4]:
pages.map(lambda x: x[0]).distinct().count()

93

### Setup

We want to capture the neighboring relationships as a list of tuples of the form `(host page, neighbor list)`

In [5]:
neighbors = pages.map(lambda x: (x[0], x[1:]))
neighbors.take(1)

[('9465097', ['12566713', '11158207', '11145916', '11883199', '12857908'])]

And initialize each page's rank to 1.0

In [6]:
ranks = neighbors.map(lambda x: (x[0], 1))
print(ranks.take(5))

[('9465097', 1), ('11581419', 1), ('8553535', 1), ('7778283', 1), ('11044077', 1)]


### Iterating

On each iteration of PageRank, we want to *send a contribution* from the host node to each of its neighbors equal to

```
rank(p)/numNeighbors(p)
```

And then set each page's rank equal `0.15 + 0.85 * contributionsReceived`.

In [7]:
neighbors.join(ranks).take(5)

[('12857908',
  (['11878901', '10233949', '12050360', '11602792', '11163673'], 1)),
 ('8957669', (['8957670', '9234794', '11145916', '9650960', '10573627'], 1)),
 ('8151305', (['7494284', '8825715', '9650960', '11160736', '9032384'], 1)),
 ('11886254', (['12518224', '12857908', '8553535', '11878901', '9650960'], 1)),
 ('12033775', (['9784363', '11287582', '8709207', '11044077', '10873782'], 1))]

In [8]:
def unpack_row(row):
    pageID, (links, rank) = row
    yield from map(lambda dest: (dest, rank / len(links)), links)

In [9]:
for i in range(10):
    contributions = neighbors.join(ranks).flatMap(unpack_row)
    ranks = contributions.reduceByKey(lambda x, y: x + y).mapValues(lambda x: 0.15 + 0.85 * x)

In [10]:
%%timeit -n1 -r1

result = ranks.collect()

2min 53s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## Tuning this with Intentional Partitioning

What type was `pages`? `neighbors`?

In [17]:
type(pages), type(neighbors)

(pyspark.rdd.PipelinedRDD, pyspark.rdd.PipelinedRDD)

Both were `RDDs`, **meaning** that each time we utilized them, they were potentially getting shuffled across the network despite never changing.

If we instead did the same implementation, but *persisted* the parts that didn't change, what kind of performance boost will we see?

In [12]:
pages = rdd.map(lambda x: x.split())
pages.persist()

PythonRDD[130] at RDD at PythonRDD.scala:48

In [13]:
neighbors = pages.map(lambda x: (x[0], x[1:]))
neighbors.persist()

PythonRDD[131] at RDD at PythonRDD.scala:48

In [14]:
ranks = pages.map(lambda x: (x[0], 1))

In [15]:
for i in range(10):
    contributions = neighbors.join(ranks).flatMap(unpack_row)
    ranks = contributions.reduceByKey(lambda x, y: x + y).mapValues(lambda x: 0.15 + 0.85 * x)

In [16]:
%%timeit -n1 -r1

result = ranks.collect()

2min 51s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


Eureka! **2 seconds of performance boost**.

# TODO

Figure why this was a bust.

Was there something I could have done? Or was it merely a limitation of running on a regular old computer?